This is a deployment on pre-trained model to recognize objects with JetBot's camera. (For Specs plz refer to the link in readme.md)

Import all the necessary libraries.

In [ ]:
from jetbot import Robot, Camera
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet18
import numpy as np
import cv2
import ipywidgets.widgets as widgets
from IPython.display import display

robot = Robot()
camera = Camera.instance(width=224, height=224)



In this project, pretrained resNet is chosen for a higher precision.

In [ ]:
model = resnet18(pretrained=True).eval()

# Create a transform to preprocess camera images and tailor it to suitable size
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

An object recognition function is the key, and the result can be used for the ethics team to rate.

In [ ]:
def recognize_objects(image):
    image_tensor = transform(image).unsqueeze(0)
    with torch.no_grad():
        outputs = model(image_tensor)
    # Get the predicted class
    _, predicted = outputs.max(1)
    return predicted.item()

This part is for a neat user-friendly interface to show the live stream and predicted items.

In [ ]:
image_widget = widgets.Image(format='jpeg', width=224, height=224)
label_widget = widgets.Text(description='Detected', width=200)

def update(change):
    image = np.array(change['new'])
    class_index = recognize_objects(image)
    
    # Here, you might convert class_index to a string using some mapping
    # depending on the dataset the model was trained on.
    
    label_widget.value = str(class_index)
    image_widget.value = cv2.imencode('.jpg', cv2.cvtColor(image, cv2.COLOR_BGR2RGB))[1].tobytes()

camera.observe(update, names='value')

display(image_widget, label_widget)

To shut down the camera and robot properly.

In [ ]:
camera.unobserve(update, names='value')
robot.stop()